In [1]:
# Install dependencies
import itertools
import random
from typing import List

import pandas as pd

from plate import Plate
from tile import Tile


In [27]:
yellow = "yellow"
red = "red"
blue = "blue"
green = "green"
black = "black"

COLORS = [red, yellow, blue, green, black]
# Test version of Board class
class Space():
    """Space on score board"""

    def __init__(self, color: str = None, minus_value: int = None):
        self.tile = None
        self.color = color
        self.minus_value = None

    @property
    def _tile(self, _tile: str):
        self.tile = _tile

    @property
    def _minus_value(self, _minus_value: int):
        self.minus_value = _minus_value
        
class Board():
    """5x5 Playing board to track game progress"""

    def __init__(self, name: str):
        self.game_over = False
        self.name = name
        self.spaces = [
            [Space(yellow), Space(blue), Space(green), Space(red), Space(black)],
            [Space(black), Space(yellow), Space(blue), Space(green), Space(red)],
            [Space(red), Space(black), Space(yellow), Space(blue), Space(green)],
            [Space(green), Space(red), Space(black), Space(yellow), Space(blue)],
            [Space(blue), Space(green), Space(red), Space(black), Space(yellow)],
            ]

    def check_rows(self):
        """
        Check if the end-game condition has been met. Game ends if
        any row on a Board is filled with tiles
        """
        for i in range(len(self.spaces)):
            has_tiles = [self.spaces[i][j].tile for j in range(len(self.spaces))]
            if all(has_tiles):
                self.game_over = True
        return

class Factory():
    """
    Board where tiles are collected by players before being moved
    to the scoring board.
    """

    def __init__(self):
        self.floor = Floor()
        self.rows = {
            "row_1": {
                "red": 1,
                "green": 1,
                "blue": 1, 
                "yellow": 1, 
                "black": 1,
            },
            "row_2": {
                "red": 2,
                "green": 2,
                "blue": 2, 
                "yellow": 2, 
                "black": 2,
            },
            "row_3": {
                "red": 3,
                "green": 3,
                "blue": 3, 
                "yellow": 3, 
                "black": 3,
            },
            "row_4": {
                "red": 4,
                "green": 4,
                "blue": 4, 
                "yellow": 4, 
                "black": 4,
            },
            "row_5": {
                "red": 5,
                "green": 5,
                "blue": 5, 
                "yellow": 5, 
                "black": 5,
            },
            "row_minus": {
                "red": 20,
                "green": 20, 
                "blue": 20, 
                "yellow": 20,
                "black": 20,
            },
        }
        self.playable_options = self.get_playable_options()

    def get_playable_options(self):
        """Convert self.rows to a pandas dataframe of all playable options"""
        options_df = pd.DataFrame(self.rows)
        options_df = options_df.reset_index()
        options_df = pd.melt(
            options_df, 
            id_vars=["index"], 
            value_vars=[x for x in options_df if x.startswith("row")]
            ).rename(columns={"index": "color", "value":"playable_spaces"})
        return options_df
            

class Floor():
    """
    Minus section of a player's Factory
    """

    def __init__(self):
        self.spaces = [
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=2),
            Space(minus_value=2),
            Space(minus_value=3),
            Space(minus_value=3),
            ]
        self._discard = []

    @property
    def discard(self):
        return self._discard

    @discard.setter
    def discard(self, val):
        self._discard = val
        
    def reset(self):
        """Reset Floor to original state"""
        for space in self.spaces:
            if space.tile:
                self.discard.append(space.tile)

        self.spaces = [
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=2),
            Space(minus_value=2),
            Space(minus_value=3),
            Space(minus_value=3),
            ]

class Player():
    """
    Represent a player in the game
    """

    def __init__(self, name=str):
        self.name = name
        self.factory = Factory()
        self.board = Board("player_1")


class Table():
    """ 
    Represents where left over tiles from plates go
    """
    
    def __init__(self, n_plates: int = 5, players=[Player]):
        colors = [red, green, blue, yellow, black]
        self._players = players
        self.n_plates = n_plates
        self.bag = [Tile(color) for color in colors for _ in range(20)]
        self.plates = {
            "plate_1": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_2": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_3": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_4": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_5": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_leftover": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
        }
    
    @property
    def players(self):
        return self._players
    
    @players.setter
    def players(self, updated_players: List[Player]):
        self._players = updated_players

    @staticmethod
    def get_discard_tiles(players):
        """Combine discard tiles from all players"""
        discard_tiles = []
        for player in players:
            discard_tiles.extend(player.factory.floor.discard)

    def fill_plates(self, discard_tiles: List[Tile]):
        """
        Fill plates with tiles from bag

        :param discard: Player's discard piles from their floors
        """
        self.empty_plates()
        n_tiles = self.n_plates * 5

        # Check if there are enough plates in the bag
        if len(self.bag) >= n_tiles:
            self.deal_plates()
        # Re-fill bag with tiles from players' minus section
        else:
            self.bag.extend(discard_tiles)
            self.deal_plates()

            # Set discard piles back to being empty for each player
            for player in self.players:
                player.factory.floor.discard = []
        self._playable_options = self.get_playable_options()

    def deal_plates(self):
        """Shuffle bag and place 4 random tiles on each plate"""
        # Shuffle tiles
        plate_names = [name for name in self.plates if name[-1].isdigit()]
        random.shuffle(self.bag)
        for plate in plate_names:
            for _ in range(4):
                tile = self.bag.pop()
                self.plates[plate][tile.color] += 1


    def empty_plates(self):
        """Reset all plates to an empty state"""
        self.plates = {
            "plate_1": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_2": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_3": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_4": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_5": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_leftover": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
        }
        self._playable_options = self.get_playable_options()

    @property
    def playable_options(self):
        return self._playable_options

    def get_playable_options(self):
        """Convert self.rows to a pandas dataframe of all playable options"""
        options_df = pd.DataFrame(self.plates)
        options_df = options_df.reset_index()
        options_df = pd.melt(
            options_df, 
            id_vars=["index"], 
            value_vars=[x for x in options_df if x.startswith("plate")]
            ).rename(columns={"index": "color", "value":"playable_tiles"})
        self._playable_options = options_df
        return self._playable_options

def game_state(table: Table, print=False):
    """Log current game state"""
    player_1 = table.players[0]
    player_2 = table.players[1]
    if print:
        print(f"Current players: {len(table.players)}")
        print(f"Player_1 factory: {player_1.factory.playable_options}")
        print(f"Player_2 factory: {player_2.factory.playable_options}")
        print()
        print(f"Player_1 discard: {player_1.factory.floor.discard}")
        print(f"Player_2 discard: {player_2.factory.floor.discard}")
        print()
        print(f"Current plate setup:")
        for plate in table.plates:
            print(f"{plate}: {table.plates[plate].items()}")
        print()
    path = "log_game_state"
    player_1.factory.playable_options.to_csv(f"{path}/player_1_playable_options.csv")
    player_2.factory.playable_options.to_csv(f"{path}/player_2_playable_options.csv")
    table.playable_options.where(table_options["playable_tiles"] != 0).dropna().to_csv(f"{path}/table_playable_options.csv")


In [33]:
# Set up game with 2 players
player_1 = Player("Kaci")
player_2 = Player("Avi")
table = Table(n_plates=5, players=[player_1, player_2])
table.fill_plates(player_1.factory.floor.discard + player_2.factory.floor.discard)

# Playable options of plates on table
table_options = table.playable_options
table_options = table_options.where(table_options["playable_tiles"] != 0).dropna()


# Playable options of player factories
player_options = player_1.factory.playable_options
player_options = player_options.where(player_options["playable_spaces"] != 0).dropna()

game_state(table)

# Get all possible options
all_options = []
for color in COLORS:
    player_condition = (player_options["color"] == color) & (player_options["playable_spaces"] > 0)
    table_condition = (table_options["color"] == color) & (table_options["playable_tiles"] > 0)
    color_options = itertools.product(player_options.loc[player_condition].index.values, table_options.loc[table_condition].index.values)
    all_options.extend(list(color_options))

print(all_options)

# Select one option
random.choice(all_options)

[(0, 0), (0, 10), (5, 0), (5, 10), (10, 0), (10, 10), (15, 0), (15, 10), (20, 0), (20, 10), (25, 0), (25, 10), (3, 3), (3, 8), (3, 18), (3, 23), (8, 3), (8, 8), (8, 18), (8, 23), (13, 3), (13, 8), (13, 18), (13, 23), (18, 3), (18, 8), (18, 18), (18, 23), (23, 3), (23, 8), (23, 18), (23, 23), (28, 3), (28, 8), (28, 18), (28, 23), (2, 12), (2, 17), (2, 22), (7, 12), (7, 17), (7, 22), (12, 12), (12, 17), (12, 22), (17, 12), (17, 17), (17, 22), (22, 12), (22, 17), (22, 22), (27, 12), (27, 17), (27, 22), (1, 1), (1, 6), (1, 11), (1, 16), (6, 1), (6, 6), (6, 11), (6, 16), (11, 1), (11, 6), (11, 11), (11, 16), (16, 1), (16, 6), (16, 11), (16, 16), (21, 1), (21, 6), (21, 11), (21, 16), (26, 1), (26, 6), (26, 11), (26, 16), (4, 9), (4, 24), (9, 9), (9, 24), (14, 9), (14, 24), (19, 9), (19, 24), (24, 9), (24, 24), (29, 9), (29, 24)]


In [26]:
player_1.factory.rows

{'row_1': {'red': 1, 'green': 1, 'blue': 1, 'yellow': 1, 'black': 1},
 'row_2': {'red': 2, 'green': 2, 'blue': 2, 'yellow': 2, 'black': 2},
 'row_3': {'red': 3, 'green': 3, 'blue': 3, 'yellow': 3, 'black': 3},
 'row_4': {'red': 4, 'green': 4, 'blue': 4, 'yellow': 4, 'black': 4},
 'row_5': {'red': 5, 'green': 5, 'blue': 5, 'yellow': 5, 'black': 5},
 'row_minus': {'red': 20, 'green': 20, 'blue': 20, 'yellow': 20, 'black': 20}}